# Customer Purchase Behavior Clustering
- This notebook performs clustering on customer purchase behavior data to identify distinct purchase patterns.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib,os

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import seaborn as sns
# pca
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score



In [ ]:

purchase_behavior_features = [
    'NumDealsPurchases',  # Promo purchases
    'NumWebPurchases',  # Online
    'NumCatalogPurchases',  # Catalog
    'NumStorePurchases',  # In-store
    'NumWebVisitsMonth'  # Website visits (optional: inverse indicator)
]

df = pd.read_csv('../featured_customer_segmentation.csv')
# Display the first few rows of the DataFrame
print("DataFrame Head:")
print(df.head())

In [ ]:
scaler = StandardScaler()
X_behavior = scaler.fit_transform(df[purchase_behavior_features])

inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_behavior)
    inertia.append(kmeans.inertia_)

print(inertia)

plt.plot(range(1, 10), inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for purchase Pattern Clustering')
plt.grid(True)
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df['PurchaseCluster'] = kmeans.fit_predict(X_behavior)

# Print cluster distributions 
cluster_counts = df['PurchaseCluster'].value_counts().sort_index()
print("\nNumber of customers in each cluster:")
print(cluster_counts)
print("\nPercentage of customers in each cluster:")
print((cluster_counts / len(df) * 100).round(2), "%")

# Plot distribution
plt.figure(figsize=(10, 6))
cluster_counts.plot(kind='bar')
plt.title('Distribution of Customers Across Clusters')
plt.xlabel('Cluster')
plt.ylabel('Number of Customers')
plt.xticks(rotation=0)
for i, v in enumerate(cluster_counts):
    plt.text(i, v, str(v), ha='center', va='bottom')
plt.tight_layout()
plt.show()


In [ ]:

# Organize features by category
feature_categories = {
    'Demographics': [ 'Education', 'Total_Dependents',
                    'Teenhome', 'Kidhome'],
    'Expenses': [ 'MntWines', 'MntFruits', 'MntMeatProducts',
                'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'],
    'Purchase Channels': ['NumWebPurchases', 'NumStorePurchases', 'NumCatalogPurchases',
                         'NumDealsPurchases','NumWebVisitsMonth'],
    'cash': ['Total_Spending','Income'],
    'channel_ratios': ['Web_Ratio', 'Store_Ratio', 'Catalog_Ratio', 'Deals_Ratio'],
    'Expense_ratios': ['Wine_Ratio', 'Fruit_Ratio', 'Meat_Ratio',
                         'Fish_Ratio', 'Sweet_Ratio', 'Gold_Ratio']
}

# Color palettes for each category
color_palettes = {
     'cash': 'Purples'  ,
    'Demographics': 'Blues',

    'Purchase Channels': 'Oranges',

    'channel_ratios': 'Reds',
    'Expenses': 'Greens',
    'Expense_ratios': 'Greys'

}


# Plot each category
for category_name, features in feature_categories.items():
    # Calculate means by cluster
    cluster_means = df.groupby('PurchaseCluster')[features].mean().round(2)
    print(f"\n{category_name.upper()} MEANS BY CLUSTER:")
    print(cluster_means)

    # Create figure with appropriate size based on feature count
    fig_width = min(14, max(10, len(features)))
    fig, ax = plt.subplots(figsize=(fig_width, 8))

    # Plot the transposed data for better visualization
    cluster_means.T.plot(kind='bar', ax=ax, colormap=color_palettes[category_name])

    # Add styling
    plt.title(f'{category_name} by Purchase Cluster', fontsize=16, pad=20)
    plt.ylabel('Average Value', fontsize=12)
    plt.xlabel('Feature', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Cluster', fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.3)

    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f', padding=3, fontsize=8)

    plt.tight_layout()
    plt.show()



In [ ]:
# Create radar chart comparing key metrics across clusters
# Select representative metrics from each category
key_metrics = ['Age', 'Income', 'Education', 'Total_Spending',
              'NumWebPurchases' ,'NumWebVisitsMonth','NumStorePurchases', 'NumCatalogPurchases',]

# Get data and normalize for radar chart
radar_data = df.groupby('PurchaseCluster')[key_metrics].mean()

# Normalize each metric to 0-100 scale
for col in radar_data.columns:
    radar_data[col] = 100 * radar_data[col] / radar_data[col].max()

# Set up radar chart
angles = np.linspace(0, 2*np.pi, len(key_metrics), endpoint=False).tolist()
angles += angles[:1]  # Close the circle

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

# Plot each cluster
for cluster in radar_data.index:
    values = radar_data.loc[cluster].tolist()
    values += values[:1]  # Close the polygon

    ax.plot(angles, values, linewidth=2, label=f'Cluster {cluster}')
    ax.fill(angles, values, alpha=0.1)

# Add labels
labels = [x.replace('_', ' ') for x in key_metrics]
labels += [labels[0]]  # Complete the circle of labels
ax.set_xticks(angles)
ax.set_xticklabels(labels)

# Customize chart
ax.set_title('Key Metrics by Purchase Cluster (Normalized)', size=16, pad=20)
ax.grid(True)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN


# Apply DBSCAN on the standardized purchase behavior data
# Adjust eps and min_samples based on your data
dbscan = DBSCAN(eps=1.0, min_samples=5)

# Fit DBSCAN model
df['DBSCAN_Cluster'] = dbscan.fit_predict(X_behavior)

# Count clusters (-1 indicates noise points)
cluster_counts = df['DBSCAN_Cluster'].value_counts().sort_index()
print("DBSCAN Cluster Counts:")
print(cluster_counts)

# Calculate percentage of data points in each cluster
total_points = len(df)
cluster_percentages = (cluster_counts / total_points * 100).round(2)
print("\nDBSCAN Cluster Percentages:")
for cluster, percentage in cluster_percentages.items():
    status = "Noise points" if cluster == -1 else f"Cluster {cluster}"
    print(f"{status}: {percentage}%")

# Apply PCA for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_behavior)

# Visualize the DBSCAN clusters with PCA
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['DBSCAN_Cluster'],
                      cmap='viridis', s=60, alpha=0.7, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='DBSCAN Cluster')
plt.title('DBSCAN Clustering of Purchase Behavior (PCA View)', fontsize=16, pad=20)
plt.xlabel(f'First Principal Component ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'Second Principal Component ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Compare DBSCAN clusters with KMeans clusters
cross_tab = pd.crosstab(df['DBSCAN_Cluster'], df['PurchaseCluster'],
                        normalize='index') * 100
print("\nCross-tabulation (%) of DBSCAN vs. KMeans clusters:")
print(cross_tab.round(2))

# Compare purchase patterns across DBSCAN clusters
purchase_metrics = purchase_behavior_features + ['Total_Purchases', 'Income', 'Total_Spending']

# Get cluster summaries, handling -1 (noise) separately
cluster_summary = df.groupby('DBSCAN_Cluster')[purchase_metrics].mean().round(2)
print("\nPurchase Behavior by DBSCAN Cluster:")
print(cluster_summary)

# Visualize key metrics by DBSCAN cluster
metrics_to_plot = ['NumWebPurchases', 'NumStorePurchases', 'NumCatalogPurchases', 'NumDealsPurchases']

plt.figure(figsize=(14, 8))
ax = cluster_summary[metrics_to_plot].plot(kind='bar', figsize=(14, 8))
plt.title('Purchase Channel Usage by DBSCAN Cluster', fontsize=16, pad=20)
plt.xlabel('DBSCAN Cluster (-1 = Noise)')
plt.ylabel('Average Number of Purchases')
plt.legend(title='Purchase Channel')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f', padding=3, fontsize=9)

plt.tight_layout()
plt.show()

# Characterize the clusters based on their features
print("\nDBSCAN Cluster Characteristics:")
for cluster in sorted(df['DBSCAN_Cluster'].unique()):
    if cluster == -1:
        print("\nNoise Points:")
    else:
        print(f"\nCluster {cluster}:")

    # Get top distinctive features
    cluster_data = cluster_summary.loc[cluster]

    # For non-noise clusters, find distinguishing features
    if cluster != -1:
        other_clusters = cluster_summary.drop(cluster)


In [ ]:


# Get average ratios per cluster
channel_ratios = df.groupby('PurchaseCluster')[['Web_Ratio', 'Store_Ratio', 'Catalog_Ratio', 'Deals_Ratio']].mean()

# Create bar plot
ax = channel_ratios.plot(kind='bar', figsize=(10, 6), width=0.8)
plt.title('Preferred Purchase Channels by Cluster')
plt.xlabel('Cluster')
plt.ylabel('Proportion of Purchases')
plt.legend(title='Channel')
plt.xticks(rotation=0)

# Add value labels on bars
for i in ax.containers:
    ax.bar_label(i, fmt='%.2f', padding=3)

plt.tight_layout()
plt.show()

# Print favorite channel for each cluster
for cluster in range(len(channel_ratios)):
    favorite_channel = channel_ratios.iloc[cluster].idxmax()
    ratio = channel_ratios.iloc[cluster][favorite_channel]
    print(f"Cluster {cluster} favorite channel: {favorite_channel.replace('_Ratio', '')} ({ratio:.2%})")


In [ ]:
# Save the KMeans model and cluster labels
os.makedirs('models', exist_ok=True)
joblib.dump(kmeans, 'models/purchase_clusters.joblib')

# Save cluster assignments to CSV 
df.to_csv('../featured_customer_segmentation_with_clusters.csv', index=False)
print("Purchase clusters model and data saved successfully")


# Purchase Cluster Interpretations

Based on the analysis, here are the three distinct customer segments based on purchase behavior:

## Cluster 0: Deal Seekers (996 customers - 44.5%)
**Characteristics:**
- **Income:** Lower average income ($35,174)
- **Education:** Slightly lower education levels  
- **Family:** Highest number of kids at home
- **Spending:** Lowest total spending ($105.60)
- **Channel Preference:** Primarily store purchases (59%) with some web purchases (33%)
- **Behavior:** Makes more web visits but fewer actual purchases - they browse and research deals

**Simple Description:** Budget-conscious families who shop carefully, prefer in-store shopping, and look for the best deals. They visit websites frequently to research but make fewer actual purchases.

## Cluster 1: Digital-First Shoppers (531 customers - 23.7%)
**Characteristics:**
- **Income:** Moderate income levels
- **Education:** Highest education levels
- **Family:** Highest number of teenagers at home
- **Channel Preference:** Highest web purchases (40%) and most web visits per month
- **Behavior:** Tech-savvy shoppers who prefer online shopping convenience

**Simple Description:** Educated, tech-savvy customers with teenagers who prefer shopping online. They are comfortable with digital channels and make frequent web purchases.

## Cluster 2: Premium Multi-Channel Buyers (713 customers - 31.8%)
**Characteristics:**
- **Income:** Highest average income ($72,875)
- **Education:** Higher education levels
- **Family:** Fewest children and teenagers at home
- **Spending:** Highest total spending ($1,117.20)
- **Channel Preference:** Uses all channels - store (44%), catalog (29%), web (27%)
- **Behavior:** Affluent customers who shop across multiple channels

**Simple Description:** High-income customers with fewer dependents who spend the most money. They shop through all channels - stores, catalogs, and online - and represent the most valuable customers.